# Quantum Variational Classifier

This tutorial discusses how a quantum computer can be used to solve a classification task.<br>
The example is based on the following paper:
<br>
<br>
<b>Supervised learning with quantum enhanced feature spaces</b><br>
Vojtech Havlicek, Antonio D. Corcoles, Kristan Temme, Aram W. Harrow, Abhinav  Kandala, Jerry M. Chow, and Jay  M. Gambetta<br>
<a href="https://arxiv.org/abs/1804.11326">arXiv:1804.11326 (2018)</a>

In [ ]:
from svm.datasets import *
from qiskit_aqua.input import get_input_instance
from qiskit_aqua import run_algorithm
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import execute
from qiskit.tools.visualization import matplotlib_circuit_drawer, circuit_drawer

from qiskit_aqua.utils.optimizers.spsa import SPSA
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# define function to draw circuits
# drawer = lambda qc: circuit_drawer(qc, basis='u1,u2,u3,id,cx,cz,h,x,ry,rz,cz')
drawer = lambda qc: matplotlib_circuit_drawer(qc, basis='u1,u2,u3,id,cx,cz,h,x,ry,rz,cz')

## Data

Before we can train the Quantum Variational Classifier, we have to load the data set that we are interested in and split it into a training and a test set.
<br>
Note that the ad hoc data set is constructed suitably for the purpose of illustration.
<br>
<br>
<b>Exercise:</b><br>
Feel free to train and test the classifier with the alternatively given training data set:<br>
<a href="https://archive.ics.uci.edu/ml/datasets/wine">UCI Machine Learning Repository - Wine Data Set</a><br>
To switch between the data sets set the <b>use_adhoc_dataset</b> to True or False.

In [ ]:
# size of training data set
training_size = 20

# size of test data set
test_size = 10

# dimension of data sets
n = 2

# construct training and test data
# set the following flag to True for the first data set and to False for the second dataset
use_adhoc_dataset = True
if use_adhoc_dataset:
    # first (artifical) data set to test the classifier
    training_input, test_input, class_labels = \
            ad_hoc_data(training_size=training_size, test_size=test_size, n=n, gap=0.3, PLOT_DATA=True)
else:
    # second data set to test the classifier
    training_input, test_input, class_labels = \
            Wine(training_size=training_size, test_size=test_size, n=n, PLOT_DATA=True)

## Quantum Feature Map

The Quantum Feature Map $\cal{U}_{\Phi(\vec{x})}$ is a quantum circuit parametrized by a classical input datapoint $\vec{x}$.
Thus, $\cal{U}$ maps $\vec{x}$ to the quantum feature space whose dimension is exponential in the number of qubits.
In this tutorial we use the following quantum feature map:
<img src="svm/feature_map.png">
<br>
where
### $$U_{\Phi(\vec{x})} = e^{i\left(x_1 Z \otimes I + x_2 I \otimes Z + x_1x_2Z \otimes Z \right)}$$

The classical data sample $\vec{x}$ is loaded by applying the Quantum Feature Map onto the initial quantum state $|0{\rangle}^{\otimes n}$:
### $$|\phi(\vec{x})\rangle = \cal{U}_{\Phi(\vec{x})}|0{\rangle}^{\otimes n}$$



In [ ]:
def feature_map(x, q):
    
    # initialize quantum circuit
    qc = QuantumCircuit(q)
    
    # apply hadamards and U_Phi twice
    for _ in range(2):
        
        # apply the hadamard and Z-rotatiion to all qubits
        for i in range(x.shape[0]):
            qc.h(q[i])
            qc.rz(2 * x[i], q[i])
        
        # apply the two qubit gate
        qc.cx(q[0], q[1])
        qc.rz(2*(np.pi-x[0])*(np.pi-x[1]), q[1])
        qc.cx(q[0], q[1])

    # return quantum circuit
    return qc

We can now use a classical vector, for instance $\begin{pmatrix} 1.5 \\ 0.3 \end{pmatrix}$ to apply the quantum feature map onto the quantum state $|0{\rangle}^{\otimes 2}$, analyze the resulting statevector and plot the corresponding quantum circuit.

In [ ]:
# initialize quantum register
num_qubits = 2
qr = QuantumRegister(num_qubits)

# initialize test data (x1, x2)
data = np.asarray([1.5, 0.3])

# get quantum circuit
qc_feature_map = feature_map(data, qr)

# draw circuit
drawer(qc_feature_map)

# simulate using local statevector simulator
job_sim = execute(qc_feature_map, 'local_statevector_simulator', shots=1)
sim_results = job_sim.result()
print('simulation: ', sim_results)
print('statevector: ', np.round(sim_results.get_statevector(), decimals=4))

## Variational Form

The variational form is similar to an artifical neural network:
<br>
The quantum circuit receives input data, processes it through multiple layers of  parametrized quantum gates and is trained by adjusting the parameters (weights).
<br>
It should be noted that in order to optimize the parameters, the outcome of the Quantum Variational Circuit must be measured. The measurement outcome corresponds to classical bits which can be integrated in an optimization algorithm, such as gradient descent. 
<br>
<br>
In this tutorial we use the following Variational Form:
<br>
<img src="svm/variational_form.png">
<br>
The single qubit rotations denoted by $\theta_{i,j}$ correspond to Z-rotations and Y-rotations and the entangler block $U_{ent}$ consists of controlled-Z gates.
Further, the parameter $d$ denotes the depth of the Variational Circuit, i.e. the number of repitions of entangler blocks and single qubit rotation layers.
<br>
<br>
<b>Exercise:</b><br>
See how the classification performance changes when you modify the types of gates used within the quantum circuit.

In [ ]:
def variational_form(q, params, depth):
    
    # initialize quantum circuit
    qc = QuantumCircuit(q)
    
    # first set of rotations
    param_idx = 0
    for qubit in range(2):
        qc.ry(params[param_idx], q[qubit])
        qc.rz(params[param_idx+1], q[qubit])
        param_idx += 2

    # entangler blocks and succeeding rotations
    for block in range(depth):
        qc.cz(q[0], q[1])
        for qubit in range(2):
            qc.ry(params[param_idx], q[qubit])
            qc.rz(params[param_idx+1], q[qubit])
            param_idx += 2
    
    # return quantum circuit
    return qc

In [ ]:
# initialize quantum register
num_qubits = 2
qr = QuantumRegister(num_qubits)

# set depth, i.e. number of entangler blocks and rotations (after the initial rotation)
depth = 2
params = [0.5,-0.3, 0.2, 0.6]*(depth+1)
qc_variational_form = variational_form(qr, params, depth)

# draw circuit
drawer(qc_variational_form)

# simulate using local statevector simulator
job_sim = execute(qc_variational_form, 'local_statevector_simulator', shots=1)
sim_results = job_sim.result()
print('simulation: ', sim_results)
print('statevector: ', np.round(sim_results.get_statevector(), decimals=4))

# The Quantum Variational Classifier

Before we can combine the previously introduced components to the quantum variational classifier, we need to introduce some more helper functions.

#### 1.  Assign a label to the measurement outcome 
We use the parity function to assign a class label to the measurement outcomes which are given as bit strings. In explicit, a measurement consisting of an even number of '1's is mapped to the first class, and a measurement outcome consisting of an odd number of '1's is mapped to the second class.

In [ ]:
def assign_label(bit_string, class_labels):
    hamming_weight = sum([int(k) for k in list(bit_string)])
    is_odd_parity = hamming_weight & 1
    if is_odd_parity:
        return class_labels[1]
    else:
        return class_labels[0]

In [ ]:
# assigns a label 
assign_label('01', class_labels)

#### 2. Map the counts of multiple shots to probabilities for the classes
After running the same circuit many times, we derive the empirical probabilities of assigning the first or second class.


In [ ]:
def return_probabilities(counts, class_labels):
    shots = sum(counts.values())
    result = {class_labels[0]: 0, 
              class_labels[1]: 0}
    for key, item in counts.items():
        label = assign_label(key, class_labels)
        result[label] += counts[key]/shots
    return result

In [ ]:
return_probabilities({'00' : 10, '01': 10}, class_labels)

#### 3. Combine everything to build the classifier
We combine the feature map, the variational circuit, and the label assignment to construct our (untrained) classifier.
<br>
Note that the classifier is already constructed in such a way that it is able to process a list of data points.

In [ ]:
def classifier_circuit(x, params, depth):
    
    q = QuantumRegister(2)
    c = ClassicalRegister(2)    
    qc = QuantumCircuit(q, c)

    qc_feature_map = feature_map(x, q)
    qc_variational_form = variational_form(q, params, depth)

    qc += qc_feature_map
    qc += qc_variational_form

    qc.measure(q, c)
    
    return qc

def classify(x_list, params, class_labels, depth=2, shots=100):
    
    qc_list = []
    for x in x_list:
        qc = classifier_circuit(x, params, depth)
        qc_list += [qc]
        
    jobs = execute(qc_list, "local_qasm_simulator", shots=shots)
    
    probs = []
    for qc in qc_list:
        counts = jobs.result().get_counts(qc)
        prob = return_probabilities(counts, class_labels)
        probs += [prob]
            
    return probs

In [ ]:
# draw classifier circuit
qc = classifier_circuit(np.asarray([0.5, 0.5]), params, depth)
drawer(qc)

# classify test data point (using random parameters constructed earlier)
x = np.asarray([[0.5, 0.5]])
classify(x, params, class_labels, depth)

#### 4. Assign a cost value to the estimation probabilities
Given a training data point and the corresponding training class label, we can calculate a cost value which represents the probability of correct/false classification. 
<br>
We only need a singificant bias towards the correct classification and, thus, use the sigmoid function to evaluate the cost value (error probability).<br>
The function is close to zero if the probability of assigning the correct class is close to one, and close to one otherwise.

In [ ]:
def cost_estimate_sigmoid(probs, expected_label):

    p = probs.get(expected_label)
    sig = None
    if np.isclose(p, 0.0):
        sig = 1
    elif np.isclose(p, 1.0):
        sig = 0
    else:
        denominator = np.sqrt(2*p*(1-p))
        x = np.sqrt(200)*(0.5-p)/denominator
        sig = 1/(1+np.exp(-x))

    return sig

In [ ]:
x = np.linspace(0, 1, 20)
y = [cost_estimate_sigmoid({'A': x_, 'B': 1-x_}, 'A') for x_ in x]
plt.plot(x, y)
plt.xlabel('Probability of assigning the correct class')
plt.ylabel('Cost value')
plt.show()

#### 5. Evaluate the overall performance for the training set
We compute the average cost value over all training data points and use this as the objective function to train our classifier.

In [ ]:
def cost_function(training_input, class_labels, params, depth=2, shots=100, print_value=False):
    
    # map training input to list of labels and list of samples
    cost = 0
    training_labels = []
    training_samples = []
    for label, samples in training_input.items():
        for sample in samples:
            training_labels += [label]
            training_samples += [sample]
        
    # classify all samples
    probs = classify(training_samples, params, class_labels, depth)
    
    # evaluate costs for all classified samples
    for i, prob in enumerate(probs):
        cost += cost_estimate_sigmoid(prob, training_labels[i])
    cost /= len(training_samples)
    
    # print resulting objective function
    if print_value:
        print('%.4f' % cost)
    
    # return objective value
    return cost

In [ ]:
cost_function(training_input, class_labels, params, depth)

## Train the classifier

Training the classifier corresponds to an optimization task. Explicitly, we want to minimize the cost value (sigmoid function) such that the classifier manages to properly label the given data.
<br>
<br>
<b>Exercise:</b><br>
See how the classification performance changes when you modify the depth of the variational form, the number of shots or number of trials.<br>
What's the best that you can achieve for the adhoc data set and the Wine data set?

In [ ]:
# set depth of variational form
depth = 2

# set number of shots to evaluate the classification circuit
shots = 100

# set number of iterations for the optimization algorithm
# (SPSA evaluates the objective function twice per iteration)
max_trials = 25

# setup the optimizer
optimizer = SPSA()
spsa_params = np.asarray([4.0, 0.1, 0.602, 0.101, 0.0])
optimizer.init_args(max_trials=max_trials, parameters=spsa_params)

# define objective function for training
objective_function = lambda params: cost_function(training_input, class_labels, params, depth, shots, print_value=True)

# randomly initialize the parameters
init_params = 2*np.pi*np.random.rand(num_qubits*(depth+1)*2)

# train classifier
opt_params, value, _ = optimizer.optimize(len(init_params), objective_function, initial_point=init_params)

# print results
print()
print('opt_params:', opt_params)
print('opt_value: ', value)

## Test the trained classifier

To check how well we could train the classifier, we evaluate the classification performance on the test data set.

In [ ]:
# collect coordinates of test data
test_label_0_x = [x[0] for x in test_input[class_labels[0]]]
test_label_0_y = [x[1] for x in test_input[class_labels[0]]]
test_label_1_x = [x[0] for x in test_input[class_labels[1]]]
test_label_1_y = [x[1] for x in test_input[class_labels[1]]]

# initialize lists for misclassified datapoints
test_label_misclassified_x = []
test_label_misclassified_y = []

# evaluate test data
for label, samples in test_input.items():
    
    # classify samples 
    results = classify(samples, opt_params, class_labels, depth, shots=shots)
    
    # analyze results
    for i, result in enumerate(results):
        
        # assign label
        assigned_label = class_labels[np.argmax([p for p in result.values()])]
        print('----------------------------------------------------')
        print('Data point:    ', samples[i])
        print('Label:         ', label)
        print('Assigned:      ', assigned_label)
        print('Probabilities: ', result)
        
        if label != assigned_label:
            print('Classification:', 'INCORRECT')
            test_label_misclassified_x += [samples[i][0]]
            test_label_misclassified_y += [samples[i][1]]
        else:
            print('Classification:', 'CORRECT')
        
# compute fraction of misclassified samples
total = len(test_label_0_x) + len(test_label_1_x)
num_misclassified = len(test_label_misclassified_x)
print()
print(100*(1-num_misclassified/total), "% of the test data was correctly classified!")

# plot results
plt.figure()
plt.scatter(test_label_0_x, test_label_0_y, c='b', label=class_labels[0], linewidths=5)
plt.scatter(test_label_1_x, test_label_1_y, c='g', label=class_labels[1], linewidths=5)
plt.scatter(test_label_misclassified_x, test_label_misclassified_y, linewidths=20, s=1, facecolors='none', edgecolors='r')
plt.legend()
plt.show()

# How to call the Quantum Variational Classifier in Qiskit Aqua

The Qiskit Aqua environment provides a method to call the Quantum Variational Classifier. 

In [ ]:
params = {
    'problem': {'name': 'svm_classification'},
    'backend': {'name': 'local_qasm_simulator', 'shots': 100},
    'algorithm': {
        'name': 'SVM_Variational',
        'circuit_depth': 3,
        'print_info': True
    },
    'optimizer': {
        'name': 'SPSA',
        'max_trials': 100,
        'save_steps': 10
    }
}

algo_input = get_input_instance('SVMInput')
algo_input.training_dataset = training_input
algo_input.test_dataset = test_input
result = run_algorithm(params, algo_input)
print(result)